# Assignment 2

#### First Name: Johnny
#### Last Name: Núñez


## 1. Load Data from JSON

In [197]:

from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Twitter Analysis")\
.getOrCreate()


In [198]:
df_twitter = spark.read.json("corona_tweet_new.json")

In [199]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- frie

In [200]:
### From the user nestec col select the following cols only id_str,followers_count,friends_count and created at 
# (2 points)
from pyspark.sql.functions import col 
df_twitter=df_twitter.select(df_twitter.created_at,
                             df_twitter.favorite_count,
                             df_twitter.hashtags,
                             df_twitter.id,
                             df_twitter.in_reply_to_status_id,
                             df_twitter.in_reply_to_user_id_str,
                             df_twitter.location,
                             df_twitter.reply_count,
                             df_twitter.retweet_count,
                             df_twitter.source,
                             df_twitter.user.id_str.alias('user_id_str'),
                             df_twitter.user.followers_count.alias('user_followers_count'),
                             df_twitter.user.friends_count.alias('user_friends_count'),
                             df_twitter.user.created_at.alias('user_created_at'))

In [201]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)



In [202]:
# Print the total count of number of records in df_twitter(1 point)
df_twitter.count()

15894

In [203]:
# Extract the source lable from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regualr expresion and the group would be 1 for this regular expression.
#(4 points)
from pyspark.sql.functions import regexp_extract, col

df_twitter=df_twitter.withColumn("extracted_source",regexp_extract('source',r'<a [^>]+>([^<]+)',1))
df_twitter.select(col('extracted_source'),col('source')).show()

+-------------------+--------------------+
|   extracted_source|              source|
+-------------------+--------------------+
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter Web Client|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
+----------

In [204]:
# Convert the DataFrame into RDD
rdd_twitter=df_twitter.rdd.map(tuple)

In [205]:
# Create a temporay table in memory with name as twitter (1 point)
df_twitter.createOrReplaceTempView("twitter")

## 2. Analyze Data

#### You will be writing code to find the answer to the questions listed below using Just RDD, Using spark SQL 

- Analyze using RDD 
- Analyze using Dataframe without temp table 
- Analyze using spark.sql with temp table


#### 2.1 Get total number of unique users (1 point for each type)


In [206]:
df_twitter

DataFrame[created_at: string, favorite_count: bigint, hashtags: array<string>, id: string, in_reply_to_status_id: string, in_reply_to_user_id_str: string, location: string, reply_count: bigint, retweet_count: bigint, source: string, user_id_str: string, user_followers_count: bigint, user_friends_count: bigint, user_created_at: string, extracted_source: string]

In [207]:
# Using RDD
rdd_twitter.map(lambda x: x[10]).distinct().count()

14094

In [208]:
# Using DataFrame
df_user_count = df_twitter.select('user_id_str').distinct()
df_user_count.count()

14094

In [209]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(DISTINCT(user_id_str)) FROM twitter').show()

+---------------------------+
|count(DISTINCT user_id_str)|
+---------------------------+
|                      14094|
+---------------------------+



#### 2.2 Get count of user who have more than 1 tweet in the data (2 points)

In [210]:
# Using RDD
rdd_twitter.map(lambda x: (x[10],1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>1).count()

1016

In [211]:
import pyspark.sql.functions as F
# Using DataFrame
# Get count of user who have more than 1 tweet in the data in df_twitter
df_twitter.groupBy('user_id_str').count().filter(F.col('count')>1).count()



1016

In [212]:
# Using spark.sql and the temporay table.
spark.sql("SELECT count(num_tweets) \
          FROM (\
            SELECT count(id) as num_tweets \
            FROM twitter \
            group by user_id_str\
          ) where num_tweets>1 ").show()

+-----------------+
|count(num_tweets)|
+-----------------+
|             1016|
+-----------------+



#### 2.3 Get total number unique extracted_source (1 point each)

In [213]:
# Using RDD
# Get total number unique extracted_source output: 133
rdd_twitter.map(lambda x: x[14]).distinct().count()

133

In [214]:
# Using DataFrame
df_source_distinct = df_twitter.select('extracted_source').distinct()
df_source_distinct.count()

133

In [215]:
# Using spark.sql and the temporay table.
spark.sql("SELECT \
           COUNT(DISTINCT(extracted_source)) \
           FROM twitter").show()

+--------------------------------+
|count(DISTINCT extracted_source)|
+--------------------------------+
|                             133|
+--------------------------------+



#### 2.4 Get top 5 most used extracted_source

In [216]:
# Using RDD (5 points)
rdd_twitter.map(lambda x: (x[14],1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1],ascending=False).take(5)


[('Twitter for Android', 6262),
 ('Twitter for iPhone', 5698),
 ('Twitter Web App', 2878),
 ('Twitter for iPad', 428),
 ('Twitter Web Client', 136)]

In [217]:
# Using DataFrame (2 points) 
df_twitter.groupBy('extracted_source').count().orderBy(F.desc('count')).limit(5).show()

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+



In [218]:
# Using spark.sql and the temporay table. (2 points)
spark.sql("SELECT \
          extracted_source, \
          COUNT(extracted_source) as count_source \
          FROM twitter \
          GROUP BY extracted_source \
          ORDER BY count_source DESC").show(5)

+-------------------+------------+
|   extracted_source|count_source|
+-------------------+------------+
|Twitter for Android|        6262|
| Twitter for iPhone|        5698|
|    Twitter Web App|        2878|
|   Twitter for iPad|         428|
| Twitter Web Client|         136|
+-------------------+------------+
only showing top 5 rows



#### 2.5 Get count of distinct hastags used ( 5 point each) 

In [219]:
# Using RDD 
rdd_twitter.flatMap(lambda x: x[2]).distinct().count()

1215

In [220]:
# Using DataFrame
df_twitter.select(F.explode(F.col('hashtags'))).distinct().count()

1215

In [221]:
# Using spark.sql and the temporay table.
spark.sql("SELECT \
          count(aux_col) \
          FROM ( \
            SELECT \
            DISTINCT(EXPLODE(hashtags)) as aux_col \
            FROM twitter \
          ) aux").show()

+--------------+
|count(aux_col)|
+--------------+
|          1215|
+--------------+



#### 2.6 Get top 5 hashtags

In [223]:
# Using RDD (4 points)
rdd_twitter.flatMap(lambda x: x[2]).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1],ascending=False).take(5)



[('طبق_القدرات_للثانويه_ياريس', 385),
 ('Corona', 319),
 ('OilPrice', 251),
 ('COVID19', 125),
 ('corona', 123)]

In [231]:
# Using DataFrame (2 points)
df_twitter.select(F.explode(F.col('hashtags')).alias('hashtags')).groupBy('hashtags').count().sort(F.col('count').desc()).head(5)            

[Row(hashtags='طبق_القدرات_للثانويه_ياريس', count=385),
 Row(hashtags='Corona', count=319),
 Row(hashtags='OilPrice', count=251),
 Row(hashtags='COVID19', count=125),
 Row(hashtags='corona', count=123)]

In [239]:
# Using spark.sql and the temporay table. (2 points)
spark.sql("SELECT \
          expl_hashtags, \
          COUNT(expl_hashtags) as count_hasthag  \
          FROM (\
            SELECT EXPLODE(hashtags) as expl_hashtags \
            FROM twitter \
          ) \
          GROUP BY expl_hashtags \
          ORDER BY count_hasthag DESC").show(5)

+--------------------+-------------+
|       expl_hashtags|count_hasthag|
+--------------------+-------------+
|طبق_القدرات_للثان...|          385|
|              Corona|          319|
|            OilPrice|          251|
|             COVID19|          125|
|              corona|          123|
+--------------------+-------------+
only showing top 5 rows



#### 2.6 Get total number of tweets which are retweeted more than 100 times

In [243]:
df_twitter

DataFrame[created_at: string, favorite_count: bigint, hashtags: array<string>, id: string, in_reply_to_status_id: string, in_reply_to_user_id_str: string, location: string, reply_count: bigint, retweet_count: bigint, source: string, user_id_str: string, user_followers_count: bigint, user_friends_count: bigint, user_created_at: string, extracted_source: string]

In [246]:
# Using RDD
rdd_twitter.filter(lambda x: x[8]>100).count()

15753

In [248]:
# Using DataFrame
df_twitter.filter(F.col("retweet_count")>100).count()


15753

In [249]:
# Using spark.sql and the temporay table.
spark.sql("SELECT \
            COUNT(id) as count_id \
            FROM twitter \
            WHERE retweet_count>100").show()

+--------+
|count_id|
+--------+
|   15753|
+--------+



#### 2.8 Get top 3 most retweeted tweets per country (8 points)

In [279]:
# Using RDD # return Get top 3 most retweeted tweets per country

# optimize the code above using groupByKey and mapValues without numpy and sort
rdd_twitter.map(lambda x: (x[6],[x[3],x[8]])).groupByKey().mapValues(lambda x: sorted(list(x),key=lambda x: x[1],reverse=True)[:3]).collect()
    

[('India',
  [['1252332114948874240', 9988],
   ['1252252336921206787', 9976],
   ['1252254519116746754', 9973]]),
 ('Pakistan',
  [['1252334264248606720', 9988],
   ['1252251912084357121', 9975],
   ['1252252126694309888', 9973]]),
 ('USA',
  [['1252331777806524416', 9994],
   ['1252254239805579264', 9987],
   ['1252335464750735362', 9982]]),
 ('Italy',
  [['1252252106750377996', 9994],
   ['1252251206027816960', 9984],
   ['1252330500670337024', 9971]]),
 ('Canada',
  [['1252335430323888128', 9997],
   ['1252254877939531776', 9992],
   ['1252252082825986051', 9987]]),
 ('China',
  [['1252335780707684352', 9998],
   ['1252253596516843520', 9993],
   ['1252255562525560832', 9984]]),
 ('Chile',
  [['1252253612140490759', 9988],
   ['1252334891951427585', 9984],
   ['1252253710182481920', 9978]]),
 ('UK',
  [['1252333018578145280', 9991],
   ['1252252091822870529', 9989],
   ['1252254043973603329', 9985]]),
 ('Mexico',
  [['1252253843145912320', 9998],
   ['1252255209776189442', 9994],
 

In [264]:
# Using DataFrame
from pyspark.sql.window import Window
df_twitter.select('id', 'location', 'retweet_count').withColumn('rt_loc_rank', F.dense_rank().over(Window.partitionBy('location').orderBy(F.desc('retweet_count')))).filter(F.col('rt_loc_rank')<=3).show()

    

+-------------------+--------+-------------+-----------+
|                 id|location|retweet_count|rt_loc_rank|
+-------------------+--------+-------------+-----------+
|1252335430323888128|  Canada|         9997|          1|
|1252254877939531776|  Canada|         9992|          2|
|1252252082825986051|  Canada|         9987|          3|
|1252253612140490759|   Chile|         9988|          1|
|1252334891951427585|   Chile|         9984|          2|
|1252253710182481920|   Chile|         9978|          3|
|1252335780707684352|   China|         9998|          1|
|1252253596516843520|   China|         9993|          2|
|1252255562525560832|   China|         9984|          3|
|1252334028092399622| Germany|         9999|          1|
|1252330902325248000| Germany|         9997|          2|
|1252252295510855682| Germany|         9990|          3|
|1252332114948874240|   India|         9988|          1|
|1252252336921206787|   India|         9976|          2|
|1252254519116746754|   India| 

In [250]:
# Using spark.sql and the temporay table.
spark.sql("SELECT * \
          FROM (\
            SELECT \
            location,\
            id,\
            retweet_count,\
            ROW_NUMBER() \
            OVER (PARTITION BY location ORDER BY retweet_count DESC) as rt_loc_rank \
            FROM twitter\
          ) aux_table \
          WHERE rt_loc_rank <= 3").show()

+--------+-------------------+-------------+-----------+
|location|                 id|retweet_count|rt_loc_rank|
+--------+-------------------+-------------+-----------+
|  Canada|1252335430323888128|         9997|          1|
|  Canada|1252254877939531776|         9992|          2|
|  Canada|1252252082825986051|         9987|          3|
|   Chile|1252253612140490759|         9988|          1|
|   Chile|1252334891951427585|         9984|          2|
|   Chile|1252253710182481920|         9978|          3|
|   China|1252335780707684352|         9998|          1|
|   China|1252253596516843520|         9993|          2|
|   China|1252255562525560832|         9984|          3|
| Germany|1252334028092399622|         9999|          1|
| Germany|1252330902325248000|         9997|          2|
| Germany|1252252295510855682|         9990|          3|
|   India|1252332114948874240|         9988|          1|
|   India|1252252336921206787|         9976|          2|
|   India|1252254519116746754| 

#### 2.9 Total number of tweets per country

In [286]:
# Using RDD (3 points)
rdd_twitter.map(lambda x: x[6]).countByValue()

defaultdict(int,
            {'India': 1480,
             'UK': 1376,
             'Spain': 1464,
             'USA': 1539,
             'Canada': 1441,
             'China': 1457,
             'Pakistan': 1470,
             'Germany': 1426,
             'Mexico': 1409,
             'Chile': 1410,
             'Italy': 1422})

In [287]:
# Using DataFrame (2 points)
df_twitter.select('location').groupBy('location').count().orderBy(F.col('count').desc()).show()

+--------+-----+
|location|count|
+--------+-----+
|     USA| 1539|
|   India| 1480|
|Pakistan| 1470|
|   Spain| 1464|
|   China| 1457|
|  Canada| 1441|
| Germany| 1426|
|   Italy| 1422|
|   Chile| 1410|
|  Mexico| 1409|
|      UK| 1376|
+--------+-----+



In [288]:
# Using spark.sql and the temporay table. (1 point)
spark.sql("SELECT \
          location,\
          COUNT(location) as tweets_per_country \
          FROM twitter \
          GROUP BY location").show()

+--------+------------------+
|location|tweets_per_country|
+--------+------------------+
| Germany|              1426|
|   India|              1480|
|   China|              1457|
|   Chile|              1410|
|   Italy|              1422|
|   Spain|              1464|
|     USA|              1539|
|  Mexico|              1409|
|      UK|              1376|
|  Canada|              1441|
|Pakistan|              1470|
+--------+------------------+



## 3. Save Data

#### 3.1 save the data such that you have seperate folder per country (2 points)

In [289]:
# Using DataFrame
df_twitter.write.partitionBy("location").format("json").save("corona_tweet_by_loc.json")

#### 3.2 Save the data as parquet files (1 points)

In [290]:
# Using DataFrame
df_twitter.write.format("parquet").save("corona_tweet_out.parquet")
    